# Depthwise-seperable 2D Convolution on GPU

## 1. Set-up 

In [1]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/gdrive/MyDrive/ece5545/user.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-ethanglaser' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-ethanglaser
M	1-conv1d_cpu.ipynb
M	2-conv1d_gpu.ipynb
M	4-gemm_gpu.ipynb
M	5-conv2d_dw_gpu.ipynb
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  src
3-conv1d_fpga.ipynb  leaderboard_id.txt     tests


## 2 Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     |████████████████████████████████| 402.4 MB 16 kB/s 


## 3. Implement `make_dwsp_conv2d_gpu_scheduler` function in `src.ops`

In that function, you are required to implemented 2D convolution and use TVM to optimize it.
Please use zero padding and unit stride. 
You can assume kernel size to be an odd number.
The padding will equals to kernel size minus ones.
In this case, the output image will preserve the input image dimension.

The `make_dwsp_conv2d_gpu_scheduler` takes following arguments:
1. Batch size $B$;
2. Input channel size $C$;
3. Input image height $H$;
4. Input image width $W$;
5. Output number of channels $O$;
6. Kernel size $K$

You should return both the TVM scheduler and the TVM opterator for 
1. Input tensor $x$ with size (B, C, H, W)
2. Input kernel weight $y$ with size (O, 1, K, K)
3. Output $out$ with size (B, O, H, W)

The scheduler should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells the usage.

In [20]:
import tvm
import torch.nn.functional as F
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_dwsp_conv2d_gpu_scheduler

B = 3
C = 4
H = 16
W = 32
K = 7
dtype = 'float32'
a_np = np.random.rand(B, C, H, W).astype(dtype)
w_np = np.random.rand(C, 1, K, K).astype(dtype)

s, inp, ker, out = make_dwsp_conv2d_gpu_scheduler(B, C, H, W, K) 
func = tvm.build(s, [inp, ker, out], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((B, C, H, W), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)
# print("a:", a)
# print("w:", w)
print("Output:", b)
print(f"2DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Output: [[[[2.43610248e-01 3.86225492e-01 1.12967896e+00 ... 1.87269521e+00
    2.85117745e+00 1.81595254e+00]
   [6.99165225e-01 1.09158170e+00 1.50708449e+00 ... 4.15108442e+00
    4.60330629e+00 3.78785515e+00]
   [5.45089185e-01 1.87890434e+00 2.57114768e+00 ... 6.18786955e+00
    6.36790371e+00 6.00405169e+00]
   ...
   [2.22712088e+00 4.66970110e+00 7.20533848e+00 ... 1.81072540e+01
    1.68811283e+01 1.57299871e+01]
   [2.88426590e+00 4.45927572e+00 6.90443516e+00 ... 1.63127480e+01
    1.63132935e+01 1.56508350e+01]
   [2.23990011e+00 4.60379601e+00 7.20234108e+00 ... 1.58918638e+01
    1.54245834e+01 1.44818449e+01]]

  [[3.02394658e-01 3.15113842e-01 1.41726539e-01 ... 1.19459188e+00
    1.06218815e+00 1.03364885e+00]
   [6.32304132e-01 1.04455972e+00 1.06595397e+00 ... 3.12817979e+00
    3.08918667e+00 3.06051040e+00]
   [8.63705695e-01 1.78838444e+00 2.04913020e+00 ... 6.58014536e+00
    6.15299511e+00 5.82519674e+00]
   ...
   [1.91130018e+00 4.48190737e+00 6.13803339e+00 

In [8]:
print(tvm.lower(s, [inp, ker, out], simple_mode=True))

@main = primfn(A_1: handle, W_1: handle, Y_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [6144], []),
             W: Buffer(W_2: Pointer(float32), float32, [196], []),
             Y: Buffer(Y_2: Pointer(float32), float32, [6144], [])}
  buffer_map = {A_1: A, W_1: W, Y_1: Y} {
  allocate(inp_pad: Pointer(global float32), float32, [10032]), storage_scope = global {
    for (b: int32, 0, 3) {
      for (c: int32, 0, 4) {
        attr [IterVar(blockIdx.y: int32, (nullptr), "ThreadIndex", "blockIdx.y")] "thread_extent" = 6;
        attr [IterVar(threadIdx.y: int32, (nullptr), "ThreadIndex", "threadIdx.y")] "thread_extent" = 4;
        for (w: int32, 0, 38) {
          if @tir.likely((((blockIdx.y*2) + floordiv(threadIdx.y, 2)) < 11), dtype=bool) {
            inp_pad_1: Buffer(inp_pad, float32, [10032], [])[(((((b*3344) + (c*836)) + (blockIdx.y*152)) + (threadIdx.y*38)) + w)]

In [17]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_dwsp_2dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-ethanglaser
============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/gdrive/MyDrive/ece5545/a3-ethanglaser, inifile:
plugins: typeguard-2.7.1
collected 1357 items                                                           

tests/test_dwsp_2dconv_gpu.py .......................................... [  3%]
.................................................